In [1]:
from __future__ import print_function, division
%run basics.ipynb

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

Using Theano backend.


In [2]:
df = pd.read_pickle('udacity_courses.df')
print(df.columns)
df.head(2)

Index([u'id', u'title', u'homepage', u'subtitle', u'level', u'starter',
       u'image', u'banner_image', u'teaser_video', u'summary',
       u'short_summary', u'required_knowledge', u'expected_learning',
       u'featured', u'syllabus', u'faq', u'full_course_available',
       u'expected_duration', u'expected_duration_unit', u'new_release',
       u'transcripts_url', u'transcripts_zip_fn', u'transcript'],
      dtype='object')


,id,title,homepage,subtitle,level,starter,image,banner_image,teaser_video,summary,...,featured,syllabus,faq,full_course_available,expected_duration,expected_duration_unit,new_release,transcripts_url,transcripts_zip_fn,transcript
0,bio110,Tales from the Genome,https://www.udacity.com/course/tales-from-the-...,Introduction to Genetics for Beginners,beginner,False,https://lh5.ggpht.com/Fh07_XbT61CGDWsQoOMQaIeH...,https://lh4.ggpht.com/_AHqr0EXcLBzF0rrwtBagE0t...,https://www.youtube.com/watch?v=sEXrjh-tme8,This course is a journey into the biology of t...,...,False,## Lesson 1: Introduction to traits and heredi...,NaN,False,3,months,False,https://www.udacity.com/course/progress#!/c-bi...,transcripts_zips/bio110.zip,In the same way that our genomes influence tr...
1,cs046,Intro to Java Programming,https://www.udacity.com/course/intro-to-java-p...,Building Programs with Classes & Objects,beginner,False,https://lh4.ggpht.com/9ytiUdz0QYHwuMJFTXcNXZn4...,https://lh3.ggpht.com/AdAALPYhCsWuIvDl0ZY6zIW2...,https://www.youtube.com/watch?v=Wsp5Rrenoq4,"In this introductory course, you'll learn and ...",...,False,"### Lesson 1: Introduction to Computers, Progr...",NaN,True,4,months,False,https://www.udacity.com/course/progress#!/c-cs046,transcripts_zips/cs046.zip,"Hi, I'm Cay. I'm Sara. We'll be the instructo..."


In [3]:
df_prereq = pd.read_csv('udacity_prereq_graph.csv')
df_prereq.head(2)

,u_id,u_pre_req_id
0,cs222,cs101
1,cs222,ph100


In [4]:
def is_prereq(a, b, preq):
    a_p = preq[preq['u_id'] == a]
    if len(a_p) > 0:
        if b in a_p['u_pre_req_id'].values.tolist():
            return True
        else:
            return False
    else:
        return False

In [7]:
fields = ['title', 'subtitle', 'summary', 'syllabus', 'transcript']
print(df[fields].count())
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=5)
all_text = df[fields[0]].copy()
for c in fields[1:]:
    all_text += ' ' + df[c]
all_text = all_text.apply(text_cleanup)
print(all_text.head(2))
vectorizer.fit(all_text)
filter_w = set(vectorizer.get_feature_names())
print('# distinct words:', len(filter_w))
print('# excluded words:', len(vectorizer.stop_words_))

title         129
subtitle      127
summary       129
syllabus      129
transcript    129
dtype: int64
0    tales from the genome introduction to genetics...
1    intro to java programming building programs wi...
Name: title, dtype: object
# distinct words: 9813
# excluded words: 30619


In [8]:
df.columns

Index([u'id', u'title', u'homepage', u'subtitle', u'level', u'starter',
       u'image', u'banner_image', u'teaser_video', u'summary',
       u'short_summary', u'required_knowledge', u'expected_learning',
       u'featured', u'syllabus', u'faq', u'full_course_available',
       u'expected_duration', u'expected_duration_unit', u'new_release',
       u'transcripts_url', u'transcripts_zip_fn', u'transcript'],
      dtype='object')

In [9]:
digiz = Digitizer()
max_words = 1000
text_series = list()
for i in fields:
    ser = df[i].copy()
    ser = ser.apply(text_cleanup, args=(filter_w, ))
    print(i, ser.apply(lambda x: len(x.split())).max())
    print(ser.head(2))
    dig_ser = digiz.series_digitizer(ser, max_len=max_words)
    text_series.append(dig_ser)
    print(dig_ser.head(2))

title 4
0        genome
1    intro java
Name: title, dtype: object
0    [1, 0, 0, 0]
1    [2, 3, 0, 0]
Name: title, dtype: object
subtitle 4
0    genetics beginners
1              programs
Name: subtitle, dtype: object
0    [121, 96, 0, 0]
1       [4, 0, 0, 0]
Name: subtitle, dtype: object
summary 69
0    journey biology genome highlight scientific so...
1    introductory essential java oriented programme...
Name: summary, dtype: object
0    [234, 235, 1, 236, 237, 123, 238, 239, 240, 24...
1    [242, 243, 3, 31, 244, 4, 3, 245, 246, 0, 0, 0...
Name: summary, dtype: object
syllabus 244
0    traits trait variation categorization inherita...
1    computers algorithms java graphics fundamental...
Name: syllabus, dtype: object
0    [241, 1219, 1220, 1221, 1222, 1223, 171, 1224,...
1    [524, 5, 3, 13, 666, 1271, 1272, 5, 1088, 1222...
Name: syllabus, dtype: object
transcript 40593
0    influence traits eye hair cancer influence dru...
1    sara instructors processor buttons secret secr...


In [10]:
text_series[0].head(2)

0    [1, 0, 0, 0]
1    [2, 3, 0, 0]
Name: title, dtype: object

In [11]:
text_series[1].head(2)

0    [121, 96, 0, 0]
1       [4, 0, 0, 0]
Name: subtitle, dtype: object

In [13]:
all_text = text_series[0].copy()
text_block_len = dict()
text_block_len[fields[0]] = len(text_series[0].iloc[0])
for i, f in zip(text_series[1:], fields[1:]):
    all_text += i
    text_block_len[f] = len(i.iloc[0])
df['digitized_text'] = all_text
max_len = df['digitized_text'].apply(len).max()
print(max_len)
df['digitized_text'] = df['digitized_text'].apply(lambda x: np.array(x))
df['digitized_text'].head(10)
for f in fields:
    print(f, 'len:', text_block_len[f])

10321
syllabus len: 244
summary len: 69
subtitle len: 4
transcript len: 10000
title len: 4


In [14]:
n_words = digiz.num_words()
print('#words:', n_words)
print('seq len:', max_len)
df['digitized_text'].head(2)
assert all(df['digitized_text'].apply(lambda x: len(x) == max_len).tolist())

#words: 9809
seq len: 10321


In [15]:
d = list()
for idx, (c_id, c_seq) in tqdm(df[['id', 'digitized_text']].iterrows(), total=len(df)):
    for jdx, (p_id, p_seq) in df[['id', 'digitized_text']].iterrows():
        if idx != jdx:
            d.append((c_id, c_seq, p_id, p_seq, is_prereq(c_id, p_id, df_prereq)))
            
X = pd.DataFrame(columns=['c_id', 'c_seq', 'p_id', 'p_seq', 'prereq'], data=d)
print(len(X))
n_pos = X['prereq'].sum()
print(n_pos)
print(len(df_prereq))
X.head(2)

100%|██████████| 129/129 [00:14<00:00,  9.01it/s]

16512
113
118


,c_id,c_seq,p_id,p_seq,prereq
0,bio110,"[1, 0, 0, 0, 121, 96, 0, 0, 234, 235, 1, 236, ...",cs046,"[2, 3, 0, 0, 4, 0, 0, 0, 242, 243, 3, 31, 244,...",False
1,bio110,"[1, 0, 0, 0, 121, 96, 0, 0, 234, 235, 1, 236, ...",cs101,"[2, 0, 0, 0, 122, 123, 0, 0, 36, 247, 122, 123...",False


In [16]:
print(X['c_seq'].apply(len).head(2))
print(X['p_seq'].apply(len).head(2))

0    10321
1    10321
Name: c_seq, dtype: int64
0    10321
1    10321
Name: p_seq, dtype: int64


In [17]:
#X['c_seq'].iloc[0]

In [18]:
X['w'] = 0
X[X['prereq'] == 1]['w'] = .5 / n_pos
X[X['prereq'] == 0]['w'] = .5 / (len(X) - n_pos)

In [39]:
def get_nn(n_words, input_len):   
    dim = 10
    emb = Embedding(input_dim=n_words, output_dim=dim, mask_zero=False)
    c = Input(shape=(max_len,), dtype='int32', name='course')
    p = Input(shape=(max_len,), dtype='int32', name='prereq')
    
    dropout = Dropout(0.5)
    c_emb = emb(c)
    p_emb = emb(p)
    
    adder = Lambda(lambda x: (x + 1.)/2., output_shape=(max_len, dim))
    c_emb = adder(emb(c))
    p_emb = adder(emb(p))
    
    #dropout = Dropout(0.1)
    #reshape = Reshape((1, max_len, dim))
    #c_do = reshape(c_emb)
    #p_do = reshape(p_emb)
    
    #seq_reduction = 1
    #maxpool = MaxPooling2D(pool_size=(seq_reduction, 1))
    #c_max = maxpool(c_do)
    #p_max = maxpool(p_do)
    #c_ac = Activation('relu')(c_emb)
    #p_ac = Activation('relu')(p_emb)
    #print(Model([c, p], p_max).summary())
    
    #axis = lambda a: len(a._keras_shape) - 1
    #dot = lambda a, b: K.batch_dot(a, b.T, axes=axis(a))
    #merge_layer = merge([c_ac, p_ac], mode=lambda x: dot(x[0], x[1]) / K.sqrt(dot(x[0], x[0]) * dot(x[1], x[1])),
                        #output_shape=lambda x: (x[0][0], x[1][0]))
    merge_layer = merge([c_emb, p_emb], mode='cos', dot_axes=2)    
    reshape_layer = Reshape((1, max_len, max_len))(merge_layer)
    
    pooling_layer = MaxPooling2D(pool_size=(40, 40))(reshape_layer)
    
    flatten_layer = Flatten()(pooling_layer)
    dense_100 = Dense(100, activation='relu')(flatten_layer)
    dense_1 = Dense(1, activation='sigmoid')(dense_100)
    
    model = Model([c, p], dense_1)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [40]:
nn = get_nn(n_words, max_len)
print(nn.summary())
#embedded_text = nn.predict(df['digitized_text'].values.tolist())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
course (InputLayer)              (None, 10321)         0                                            
____________________________________________________________________________________________________
prereq (InputLayer)              (None, 10321)         0                                            
____________________________________________________________________________________________________
embedding_8 (Embedding)          (None, 10321, 10)     98090       course[0][0]                     
                                                                   prereq[0][0]                     
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 10321, 10)     0           embedding_8[2][0]       

In [41]:
#print(embedded_text.shape)

In [42]:
# np.array([np.array(i) for i in X['c_seq'].values]).shape
print(len(X))

16512


In [43]:
def data_gen():
    idx = X.index
    while True:
        #sample_idx = np.random.choice(idx, replace=True, size=30000)
        pos_idx = X[X['prereq']>0].index
        neg_idx = X[X['prereq']==0].index
        sample_idx = list()
        sample_idx += list(pos_idx)
        sample_idx += list(np.random.choice(neg_idx, replace=False, size=len(pos_idx)*3))
        #sample_idx = list(idx)
        random.shuffle(sample_idx)
        samples = X.loc[sample_idx]
        samples_y = samples['prereq'].values
        w = np.zeros(len(samples_y))
        pos_s = samples_y.sum()
        neg_s = len(samples_y) - pos_s
        weights = len(w)/2.
        w[samples_y==1] = weights/pos_s
        w[samples_y==0] = weights/neg_s
        print('pos', samples['prereq'].values.sum())
        print(samples.count())
        for i in samples['p_seq'].values:
            [int(j) for j in i]
        #print('')
        #print('')
        yield [np.vstack(samples['c_seq'].values), np.vstack(samples['p_seq'].values)], samples_y, w, samples.index

In [44]:
x_in, y_in, w_in, x_idx = next(data_gen())
print(x_in[0].shape)
print(x_in[1].shape)
print(y_in.shape)
print(y_in.dtype)
print(w_in.shape)
hist = nn.fit(x_in, y_in, sample_weight=w_in,
              batch_size=32,
              validation_split=.1,
              shuffle=True,
              nb_epoch=10, 
              verbose=1)

data = zip(hist.history['loss'], hist.history['val_loss'])
hist_df = pd.DataFrame(columns=['train', 'val'], data=data[1:])
hist_df.plot(y=['train', 'val'], secondary_y=['val'])
plt.show()

pos 113
c_id      452
c_seq     452
p_id      452
p_seq     452
prereq    452
w         452
dtype: int64
(452, 10321)
(452, 10321)
(452,)
bool
(452,)
Train on 406 samples, validate on 46 samples
Epoch 1/10
406/406 [==============================] - 328s - loss: 0.6996 - acc: 0.2562 - val_loss: 2.3809 - val_acc: 0.1957
Epoch 2/10
406/406 [==============================] - 333s - loss: 0.6988 - acc: 0.2562 - val_loss: 2.2924 - val_acc: 0.1957
Epoch 3/10
406/406 [==============================] - 325s - loss: 0.6988 - acc: 0.2562 - val_loss: 2.2383 - val_acc: 0.1957
Epoch 4/10
406/406 [==============================] - 332s - loss: 0.6989 - acc: 0.2562 - val_loss: 2.1219 - val_acc: 0.1957
Epoch 5/10
406/406 [==============================] - 338s - loss: 0.6988 - acc: 0.2562 - val_loss: 2.0689 - val_acc: 0.1957
Epoch 6/10
406/406 [==============================] - 324s - loss: 0.6988 - acc: 0.2562 - val_loss: 2.0600 - val_acc: 0.1957
Epoch 7/10
406/406 [==============================] - 3

KeyboardInterrupt: 

In [ ]:
y_pred = nn.predict(x_in)
X['prediction'] = 0.
X.loc[x_idx, 'prediction'] = y_pred
sns.violinplot(x='prereq', y='prediction', data=X.loc[x_idx])
plt.show()
t = X.loc[x_idx][['c_id', 'p_id', 'prereq', 'prediction']]
t[t['prereq'] == False].sort_values(by='prediction', ascending=False)